# LiDAR-thermal camera extrinsinc calibration
## Data preparation

The root folder should contain a folder with the lidar scans and a folder with the thermal images:
- The LiDAR scans are saved as `.bin` files where we assume the data is arranged as `xyzi`. The file name of each scan is the timestamp.
- Thermal images are 8-bit png files, where each image name is its timestamp.
- A `timestamps.csv` should contain the LiDAR timestamps with corresponding thermal image timestamps. 

All intermediate results will be stored in the same root folder.

In [ ]:

import os
import csv
import numpy as np


# Provide the paths below:
root = "/dataset"
camera_path = os.path.join(root, "Boson")
lidar_path = os.path.join(root, "Velodyne")

# Initialize an empty dictionary
timestamp_dict = {}
time_diffs = []

# Read the CSV file
with open(os.path.join(root, 'timestamps.csv'), mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        value, key = row  # thermal camera timestamps used as key
        timestamp_dict[key] = value
        time_diffs.append(np.abs(float(key) - float(value)) *1e-6)

N_stamps = len(timestamp_dict)
mean_time_diff = np.mean(time_diffs)
std_time_diff = np.std(time_diffs)
print('Mean time difference:', mean_time_diff, 'ms')
print('Std time difference:', std_time_diff, 'ms')

inv_timestamp_dict = {} # lidar key, thermal value
for key in timestamp_dict:
    new_key = timestamp_dict[key]
    inv_timestamp_dict[new_key] = key

## 2D Pattern Extraction and Thermal Pose Estimation
### Prepare detector

In [ ]:
def transposeT(Tmatrix):
    
    out = np.eye(4)
    out[:3, :3] = Tmatrix[:3, :3].T
    out[:3, 3] = -out[:3, :3].dot(Tmatrix[:3, 3])
    
    return out

import cv2
import numpy as np
import matplotlib.pyplot as plt
Cmat = np.array([403.86291415, 0., 315.86709071, 0., 403.77831323, 249.0262173, 0.0, 0.0, 1.0]).reshape((3,-1))
dist_coeffs = np.array([-0.38149332, 0.16357154, -0.00070473,  0.00065493, -0.03794994])
print(Cmat)

pattern_size = (4, 11) # Grid size of the calibration board

params = cv2.SimpleBlobDetector_Params()   
# Change thresholds
params.minThreshold = 10 #50
params.maxThreshold = 3000 #1500

# Filter by Area
params.filterByArea = True
params.minArea = 1 #15
params.maxArea = 5000

# Filter by Circularity
params.filterByCircularity = True
params.minCircularity = 0.01
params.maxCircularity = 30.0 #15.0

# Filter by Convexity
params.filterByConvexity = True
params.minConvexity = 0.01
params.maxConvexity = 30.0 #15.0

# Filter by Inertia
params.filterByInertia = True
params.minInertiaRatio = 0.01

detector = cv2.SimpleBlobDetector_create(params)

### Iterate over the thermal images to estimate the 3D poses of the thermal camera

In [ ]:
from grid3D import *
from scipy.spatial.transform import Rotation as R

headers = ["timestamp", "angle [deg]"] + [f"T_thermal_w_{i}" for i in range(16)]
calib_data2D = []
save_path_imgs = os.path.join(root, '2D_patterns')
os.makedirs(save_path_imgs, exist_ok=True)
save_path_points = os.path.join(root, '2D_points')
os.makedirs(save_path_points, exist_ok=True)

grid_pts3d = get_asymmetric_grid_3D(lidar_mode=True, flip_start_end=True) * 0.0424 
# Add bottom_right corner:
bottom_right = grid_pts3d[0,:] + np.array([0.0, 0.0629, -0.3535])
# grid_pts3d = np.vstack([grid_pts3d, bottom_right])

# Center around bottom right and make y positive to the left (in line with calibration board planes and Velodyne) 

grid_pts3d -= bottom_right
grid_pts3d[:,1] *= -1.0
np.savetxt(os.path.join(root, 'grid3D.csv'), grid_pts3d, fmt='%.4f', delimiter=',', header='x, y, z', comments='')

# Threshold mask
thr = 135

T_rot_z_180 = np.array([[-1., 0., 0., 0.], [0., -1., 0., 0.,], [0., 0., 1., 0.], [0., 0., 0., 1.]]) # bring to world frame

for i, key in enumerate(timestamp_dict):

    img_file = key + '.png'
    img = cv2.imread(os.path.join(camera_path, img_file),0)

    _, mask = cv2.threshold(img, thr, 255, cv2.THRESH_BINARY_INV) #THRESH_BINARY_INV
    found, centers = cv2.findCirclesGrid(mask, pattern_size, None, flags = (cv2.CALIB_CB_ASYMMETRIC_GRID + + cv2.CALIB_CB_CLUSTERING), blobDetector=detector)

    disp_img = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
    if found:
        # Undistort 2D-grid
        centers_u = cv2.undistortPoints(centers, Cmat, dist_coeffs)
        centers_u = cv2.convertPointsToHomogeneous(centers_u) @ Cmat.T
        centers_u = centers_u[:,0,:2]

        # Compute 2D angle pattern
        angle = np.arctan2(centers_u[0,1] - centers_u[-4,1], centers_u[0,0] - centers_u[-4,0]) / np.pi * 180.0

        # Display results
        disp_img = cv2.drawChessboardCorners(disp_img, pattern_size, centers, found)

        # Compute 2D pose
        _, rvec, tvec = cv2.solvePnP(grid_pts3d, centers, Cmat, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)
        R_thermal, _ = cv2.Rodrigues(rvec)

        T_thermal = np.eye(4)
        T_thermal[:3,:3] = R_thermal
        T_thermal[:3,3] = tvec[:,0]
        T_thermal_w = transposeT(T_thermal)

        new_entry = [key, angle] + T_thermal_w.flatten().tolist()
        calib_data2D.append(new_entry)
        cv2.imwrite(os.path.join(save_path_imgs, img_file), disp_img)
        
        np.savetxt(os.path.join(save_path_points, key+'.csv'), centers_u, fmt='%.3f', delimiter=',', header='u, v', comments='')

    progress = float(i) / float(N_stamps) * 100.0
    print("Progress:", progress)

# Write to CSV
with open(os.path.join(root,'poses2D.csv'), mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(headers)  # Write headers
    writer.writerows(calib_data2D)  # Write data rows

## 3D Planes extraction and LiDAR Pose Estimation
### Function definitions for 3D LiDAR pose estimation

In [ ]:
import random
import time

from planeExtraction import *
from planes3D_tools import *

save_path_imgs_3D = os.path.join(root, '3D_planes')
os.makedirs(save_path_imgs_3D, exist_ok=True)
save_path_clouds = os.path.join(root, '3D_board_points')
os.makedirs(save_path_clouds, exist_ok=True)    

def get_orthonormal_rotation(Rmat):

    # Step 1: SVD
    U, _, Vt = np.linalg.svd(Rmat)

    # Step 2: Compose corrected rotation matrix
    R_fixed = U @ Vt

    # Step 3: Ensure determinant is +1 (not -1 due to reflection)
    if np.linalg.det(R_fixed) < 0:
        U[:, -1] *= -1
        R_fixed = U @ Vt

    return R_fixed

def extract_3d_board(stamp2D, angle2D):
    
    lidar_file = timestamp_dict[stamp2D] + '.bin'
    lidar = np.fromfile(os.path.join(lidar_path, lidar_file), dtype=np.float32).reshape(-1, 4)

    points = lidar[:,:3]
    points = prefilter_pointcloud(points, 45, 3.0)

    points = RemoveNoiseStatistical(points, nb_neighbors=50, std_ratio=0.2)

    results = DetectMultiPlanes(points, min_ratio=0.15, threshold=0.02, iterations=2000)
    # # print('Time:', time.time() - t0)
    planes = []
    colors = []
    plane_eqs = []
    centers = []

    for w, plane in results:

        r = random.random()
        g = random.random()
        b = random.random()

        color = np.zeros((plane.shape[0], plane.shape[1]))
        color[:, 0] = r # 0.5
        color[:, 1] = g # 0.5
        color[:, 2] = b # 0.5
            
        planes.append(plane)
        colors.append(color[:plane.shape[0],:])
        plane_eqs.append(w)


    # print("Detected {0} planes".format(len(planes)))
    if len(planes) < 3:
        return []

    closest_idxs = find_3_closest_planes(planes)
    planes = [planes[i] for i in closest_idxs]
    colors = [colors[i] for i in closest_idxs]
    plane_eqs = [plane_eqs[i] for i in closest_idxs]

    # Remove outlier planes:
    skip_scan = False
    for plane in planes:
        # print(len(plane))
        if len(plane) < 100:
            # print("Skip this scan")
            skip_scan = True

    if skip_scan:
        return []

    planes = np.concatenate(planes, axis=0)
    new_planes, new_colors = refine_plane_boundaries(planes, plane_eqs)
    for plane in new_planes:
        center = np.mean(plane, axis=0)
        centers.append(center)

    board_center = np.mean(np.array(centers), axis=0)

    new_planes = removeNoisePlanes(new_planes, centers, 0.35)

    plane_order = find_plane_order(plane_eqs, centers)
    if angle2D > 45.0:
        plane_order = shift_to_left(plane_order)

    # Rearrange in order target plane first
    new_planes, new_colors, plane_eqs = reorder_planes(new_planes, plane_eqs, plane_order)

    point12, line12 = plane_intersection(plane_eqs[0][:3], plane_eqs[0][3], plane_eqs[1][:3], plane_eqs[1][3])
    point13, line13 = plane_intersection(plane_eqs[0][:3], plane_eqs[0][3], plane_eqs[2][:3], plane_eqs[2][3])
    if line12[2]<0:
        # Make the direction of this axis go from left to right according to the calibration pattern x-direction
        line12 = -line12
    if line13[1]<0:
        # Make the direction of this axis go from right to left according to the calibration board planes
        line13 = -line13

    line12 = line12 / np.linalg.norm(line12)
    line13 = line13 / np.linalg.norm(line13)

    check_orthogonality = np.abs(np.dot(line12, line13))
    # print('Orthogonality: ', check_orthogonailty)
    if check_orthogonality > 1e-1:
        # print(stamp2D, ': No orthogonal axes 12 and 13')
        return []

    line23 = np.cross(line12, line13)
    if line23[0]<0:
        # Make the direction of this axis go from left to right according to the calibration pattern x-direction
        line23 = -line23

    line23 = line23 / np.linalg.norm(line23)

    intersection_pt = line_intersection(point12, line12, point13, line13)
    if np.linalg.norm(intersection_pt - board_center) > 0.5:
        return []

    target_orientation = np.array([line23.T, line13.T, line12.T]).T
    target_orientation_orthonormal = get_orthonormal_rotation(target_orientation)

    T_lidar = np.eye(4)
    T_lidar[:3,:3] = target_orientation_orthonormal
    T_lidar[:3,3] = intersection_pt
    T_lidar_w = transposeT(T_lidar)
    new_entry = [timestamp_dict[stamp2D]] + T_lidar_w.flatten().tolist()

    viz_line12 = viz_line_points(intersection_pt, line12, offset=0.75, dir_only=1)
    viz_line13 = viz_line_points(intersection_pt, line13, offset=0.75, dir_only=1)
    viz_line23 = viz_line_points(intersection_pt, line23, offset=0.75, dir_only=1) 


    colors = ['r', 'g', 'b']
    # Create a figure
    fig = plt.figure(1)

    # Add a 3D subplot
    ax = fig.add_subplot(111, projection='3d')

    for i in range(3):
        # indices = np.random.choice(new_planes[i].shape[0], 1000, replace=False)
        viz_pts = new_planes[i][:,:]
        ax.scatter(viz_pts[:,0], viz_pts[:,1], viz_pts[:,2], c=colors[i])

    ax.plot3D(viz_line12[:,0], viz_line12[:,1], viz_line12[:,2])
    ax.plot3D(viz_line13[:,0], viz_line13[:,1], viz_line13[:,2])
    ax.plot3D(viz_line23[:,0], viz_line23[:,1], viz_line23[:,2])
    ax.scatter(intersection_pt[0], intersection_pt[1], intersection_pt[2], c='k', s=50)

    # Add labels
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')
    ax.set_zlabel('z [m]')

    set_axes_equal(ax, set_limits=True, axis_limits=[[0.4, 2.6], [-1.5, 1.5], [-1.5, 0.8]])
    ax.view_init(elev=45, azim=-180)

    img_name = timestamp_dict[stamp2D] + '.png'
    plt.savefig(os.path.join(save_path_imgs_3D, img_name), dpi=300, bbox_inches='tight')
    plt.close()

    with open(os.path.join(save_path_clouds, timestamp_dict[stamp2D] + '.csv'), mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(['x', 'y', 'z', 'plane'])
        for i in range(3):
            # for points in new_planes[i]:
            points = new_planes[i]
            for j in range(0,len(points), 5):
                new_point = [points[j,0], points[j,1], points[j,2], i]
                writer.writerow(new_point)

    return new_entry

            

### Iterate over the LiDAR scans to estimate the 3D poses of the LiDAR

In [ ]:
headers3D = ["timestamp"] + [f"T_lidar_w_{i}" for i in range(16)]
calib_data3D = []

patterns2D = np.genfromtxt(os.path.join(root, 'poses2D.csv'), delimiter=',', skip_header=1)
stamps2D = patterns2D[:,0].astype(int).astype(str) # Only timestamp and angle for now
angles2D = patterns2D[:,1]

N_stamps = len(stamps2D)


for i, stamp2d in enumerate(stamps2D):
    pose3D = extract_3d_board(stamp2d, angles2D[i])
    progress = float(i) / float(len(stamps2D)) * 100.0
    print('Progress: ', progress)
    if len(pose3D) > 0:
        calib_data3D.append(pose3D)


# Write to CSVs
with open(os.path.join(root,'poses3D.csv'), mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(headers3D)  # Write headers
    writer.writerows(calib_data3D)  # Write data rows

## Update camera pose list

Only include camera poses that have a corresponding LiDAR pose.

In [ ]:
Lidar_stamps = np.genfromtxt(os.path.join(root, 'poses3D.csv'), delimiter=',', skip_header=1)[:,0].astype(int).astype(str)
cam_poses = np.genfromtxt(os.path.join(root, 'poses2D.csv'), delimiter=',', skip_header=1)
cam_stamps = cam_poses[:,0].astype(int).astype(str)

update_cam_poses = []
for i, cam_stamp in enumerate(cam_stamps):
    query_lidar_stamp = timestamp_dict[cam_stamp]
    if query_lidar_stamp in Lidar_stamps:
        new_entry = [cam_stamp] + cam_poses[i,2:].tolist()
        update_cam_poses.append(new_entry)
    
# Write to CSVs
headers2D = ["timestamp"] + [f"T_thermal_w_{i}" for i in range(16)]
with open(os.path.join(root,'poses2D_updated.csv'), mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(headers2D)  # Write headers
    writer.writerows(update_cam_poses)  # Write data rows

## Optimization

### Residual function

Cycle constraint, linking camera poses, LiDAR poses (at two random timestamps) and the extrinsic transformation together (rigidity).

In [ ]:
import jax
import jax.numpy as jnp
from jax import lax
from jaxlie import SE3
from scipy.spatial.transform import Rotation as R

def transposeT_jax(Tmatrix):
    
    out = jnp.eye(4)
    out = out.at[:3, :3].set(Tmatrix[:3, :3].T)
    out = out.at[:3, 3].set(-out[:3, :3] @ Tmatrix[:3, 3])
    
    return out

def rodrigues_to_matrix(r):
    rot = R.from_rotvec(r)
    return rot.as_matrix()

def rodrigues_to_matrix_jax(r):
    theta = jnp.linalg.norm(r)
    def near_zero():
        return jnp.eye(3)

    def normal_case():
        r_hat = r / theta
        K = jnp.array([
            [0, -r_hat[2], r_hat[1]],
            [r_hat[2], 0, -r_hat[0]],
            [-r_hat[1], r_hat[0], 0]
        ])
        return jnp.eye(3) + jnp.sin(theta) * K + (1 - jnp.cos(theta)) * (K @ K)

    return jax.lax.cond(theta < 1e-8, near_zero, normal_case)

def matrix_to_rodrigues(Rmat):
    rot = R.from_matrix(Rmat)
    return rot.as_rotvec()

def matrix_to_rodrigues_jax(Rmat):
    trace = jnp.trace(Rmat)
    cos_theta = (trace - 1.0) / 2.0
    cos_theta = jnp.clip(cos_theta, -1.0, 1.0)
    theta = jnp.arccos(cos_theta)

    def near_zero():
        return jnp.zeros(3)

    def general_case():
        denom = jnp.sin(theta) + 1e-8
        skew_sym = (Rmat - Rmat.T) / (2.0 * denom)
        return theta * jnp.array([
            skew_sym[2,1],
            skew_sym[0,2],
            skew_sym[1,0]
        ])

    return jax.lax.cond(theta < 1e-6, near_zero, general_case)

def cartesian_to_spherical_np(xyz):

    r = np.sqrt(xyz[0]**2 + xyz[1]**2 + xyz[2]**2)
    theta = np.arctan2(xyz[1], xyz[0])              # azimuthal angle
    phi = np.arccos(xyz[2] / r) if r != 0 else 0  # polar angle

    return np.array([r, theta, phi])

def cartesian_to_spherical_jax(xyz):

    r = jnp.sqrt(xyz[0]**2 + xyz[1]**2 + xyz[2]**2)
    theta = jnp.arctan2(xyz[1], xyz[0]) # azimuthal angle
    z_clamped = jnp.clip(xyz[2] / r, -1.0 + 1e-6, 1.0 - 1e-6)
    phi = jnp.arccos(z_clamped)  # Polar angle
    
    return jnp.array([r, theta, phi])

def spherical_to_cartesian_np(rtp):
    
    r, theta, phi = rtp

    x = r * np.sin(phi) * np.cos(theta)
    y = r * np.sin(phi) * np.sin(theta)
    z = r * np.cos(phi)

    return np.array([x, y, z])

def spherical_to_cartesian_jax(rtp):

    r, theta, phi = rtp
    
    x = r * jnp.sin(phi) * jnp.cos(theta)
    y = r * jnp.sin(phi) * jnp.sin(theta)
    z = r * jnp.cos(phi)

    return jnp.array([x, y, z])

def pose_vec_to_matrix_jax(pose_vec):
    
    out = jnp.eye(4)
    out = out.at[:3,:3].set(rodrigues_to_matrix_jax(pose_vec[:3]))
    out = out.at[:3,3].set(pose_vec[3:])
    
    return out

def se3_log_jax(T):
    return SE3.from_matrix(T).log()

def cycle_constraint_residual(camera_pose1, camera_pose2, lidar_pose1, lidar_pose2, extrinsic, rot_weight=1.0, trans_weight=1.0):
    
    R_ex = rodrigues_to_matrix_jax(extrinsic[:3])
    t_ex = extrinsic[3:] # spherical_to_cartesian_jax(extrinsic[3:])
    
    T_l_c = jnp.eye(4)
    T_l_c = T_l_c.at[:3,:3].set(R_ex)
    T_l_c = T_l_c.at[:3,3].set(t_ex)
    T_c_l = transposeT_jax(T_l_c)

    rel_pose_cam_1_2 = transposeT_jax(camera_pose2) @ camera_pose1
    rel_pose_lidar_2_1 = transposeT_jax(lidar_pose1) @ lidar_pose2

    res_mat =  rel_pose_lidar_2_1 @ (T_c_l @ (rel_pose_cam_1_2 @ T_l_c)) # If perfect extrinsinc calibration, this should return an Identity matrix

    se3_vec = se3_log_jax(res_mat)

    residual = rot_weight * jnp.linalg.norm(se3_vec[:3]) + trans_weight * jnp.linalg.norm(se3_vec[:3])
    
    return residual

def make_total_cost_function_and_grad(cam_poses, lidar_poses, pair_idxs):

    def total_cost_fn(x):
        
        x = jnp.array(x)
        
        batched_cycles = jax.vmap(cycle_constraint_residual, in_axes=(0, 0, 0, 0, None))
        cycle_res = jnp.sum(batched_cycles(cam_poses, cam_poses[pair_idxs,:,:], lidar_poses, lidar_poses[pair_idxs,:,:], x))
        
        return cycle_res / cam_poses.shape[0]
    
    cost_fn_jit = jax.jit(total_cost_fn)
    grad_fn_jit = jax.jit(jax.grad(total_cost_fn))

    return cost_fn_jit, grad_fn_jit

### Define Optimizer class

In [ ]:
import jax.numpy as jnp
from scipy.optimize import minimize

class ExtrinsicCalibrationOptimizer:
    
    def __init__(self, cam_poses, lidar_poses):
        
        # Camera parameters, detected 2d and 3D points of the calibration board
        self.cam_poses = jnp.array(cam_poses)
        
        # LiDARs parameters, scans and point matches:
        self.lidar_poses = jnp.array(lidar_poses)
        
        # Indices to extract the different sensor poses and extrinsic parameters from the 1-dimensional optimized parameter vector.
        # First n_cam_poses * 6 parameters are camera poses, the next n_lidar_poses * 6 parameters are lidar poses and the final 6 parameters are the extrinsic parameters:
        self.n_cam_poses = self.cam_poses.shape[0]
        self.n_lidar_poses =  self.lidar_poses.shape[0]

        self.pair_idxs = np.random.randint(0,self.n_cam_poses,self.n_cam_poses)

    def evaluate(self, x):

        cost_fn, grad_fn = make_total_cost_function_and_grad(self.cam_poses, self.lidar_poses)
        print('Cost: ', cost_fn(x))
        print('Grad: ', grad_fn(x))

        
    def estimate(self, init_extrinsic=np.zeros(6)):
        
        cost_fn, grad_fn = make_total_cost_function_and_grad(self.cam_poses, self.lidar_poses, self.pair_idxs)
        
        x0 = init_extrinsic 
        print('x0: ', x0)
        
        results = minimize(
            fun=lambda x: float(cost_fn(x)),
            x0=x0,
            jac=lambda x: np.array(grad_fn(x)),
            method='L-BFGS-B',
            options={'disp': True,
                     'ftol': 1e-12,
                     'gtol':1e-12,
                     'maxiter': 200}
        )
        
        print(results)
        final_extrinsic_vec = results.x
        
        return final_extrinsic_vec
    
    def evalutate_std_error(self, final_extrinsic=np.zeros(6)):
        
        T_l_c_diffs = []

        T_l_c_final = np.eye(4)
        T_l_c_final[:3,:3] = rodrigues_to_matrix(final_extrinsic[:3])
        T_l_c_final[:3,3] = final_extrinsic[3:]

        for i in range(self.n_cam_poses):
            progress = float(i) / float(self.n_cam_poses) * 100.0
            print("Progress: ", progress)

            cost_fn, grad_fn = make_total_cost_function_and_grad(jnp.expand_dims(self.cam_poses[i], axis=0),
                                                                 jnp.expand_dims(self.lidar_poses[i], axis=0), 
                                                                 jnp.expand_dims(self.pair_idxs[i], axis=0))
        
            x0 = final_extrinsic
            
            results = minimize(
                fun=lambda x: float(cost_fn(x)),
                x0=x0,
                jac=lambda x: np.array(grad_fn(x)),
                method='L-BFGS-B',
                options={'disp': True,
                        'ftol': 1e-12,
                        'gtol':1e-12,
                        'maxiter': 200}
            )
        
            T_l_c_est = np.eye(4)
            T_l_c_est[:3,:3] = rodrigues_to_matrix(results.x[:3])
            T_l_c_est[:3,3] = results.x[3:]
            T_l_c_diffs.append(T_l_c_final - T_l_c_est)

        T_l_c_diffs_np = np.array(T_l_c_diffs)
        sqrd_errors = T_l_c_diffs_np**2
        sum_sqrd_errors = np.sum(sqrd_errors, axis=0)
        std_error = np.sqrt(sum_sqrd_errors / T_l_c_diffs_np.shape[0]) / np.sqrt(T_l_c_diffs_np.shape[0])

        print(std_error)
        return std_error

### Prepare observations for optimization

In [ ]:
cam_poses = np.genfromtxt(os.path.join(root, 'poses2D_updated.csv'), delimiter=',', skip_header=1)
lidar_poses = np.genfromtxt(os.path.join(root, 'poses3D.csv'), delimiter=',', skip_header=1)

cam_pose_mats = []
lidar_pose_mats = []

for i in range(cam_poses.shape[0]):
    Tc = cam_poses[i,1:].reshape(4,4)
    Tc[:3,3]
    cam_pose_mats.append(Tc)

    Tl = lidar_poses[i,1:].reshape(4,4)
    Tl[:3,3]
    lidar_pose_mats.append(Tl)

cam_pose_mats = np.array(cam_pose_mats)
lidar_pose_mats = np.array(lidar_pose_mats)

### Running the optimizing

In [ ]:
optimizer = ExtrinsicCalibrationOptimizer(cam_pose_mats, lidar_pose_mats)

extrinsic_guess = np.array([1.20919958, -1.20919958,  1.20919958, 0.03, -.03, -0.077])

final_extrinsic = optimizer.estimate(extrinsic_guess)

### Save the obtained extrinsic parameters

In [ ]:
T_l_c_final = np.eye(4)
T_l_c_final[:3,:3] = rodrigues_to_matrix(final_extrinsic[:3])
T_l_c_final[:3,3] = final_extrinsic[3:]

print('Final T_l_c:')
print(np.array2string(T_l_c_final, precision=4, suppress_small=True))


## Evaluation

Compute the deviation between the general solution and the solution to each pair of thermal-LiDAR measurements.

In [ ]:
optimizer = ExtrinsicCalibrationOptimizer(cam_pose_mats, lidar_pose_mats)

T_l_c_final = np.genfromtxt(os.path.join(root, 'Final_T_l_c.txt'))
final_ext_vec = np.concatenate([matrix_to_rodrigues(T_l_c_final[:3,:3]), T_l_c_final[:3,3]])

std_error = optimizer.evalutate_std_error(final_ext_vec)
np.savetxt(os.path.join(root, 'std_error_extrinsics.txt'), std_error, fmt='%.8f')

### Projection

Project all LiDAR scans into their respective thermal image using the obtained extrinsic transformation.

In [ ]:
# %matplotlib qt

import matplotlib.pyplot as plt
from scipy.ndimage import minimum_filter
import time

def projectToImg(points3D):

    points2D = []
    for pt3D in points3D:

        if pt3D[2] < 0.3 or pt3D[2] > 10.0:
            continue

        u = np.round(Cmat[0,0] * pt3D[0] / pt3D[2] + Cmat[0,2])
        v = np.round(Cmat[1,1] * pt3D[1] / pt3D[2] + Cmat[1,2])

        if (u < 0) or (u >= 640) or (v < 0) or (v >= 512) or np.isnan(u) or np.isnan(v):
            continue

        points2D.append([u,v,pt3D[2]])

    return np.array(points2D)

def check_visibility(projected_points):
    
    depth_map = np.full((512, 640), np.inf)
    visibility_mask = []
    
    for i, uvd in enumerate(projected_points):
        u = np.round(uvd[0]).astype(int)
        v = np.round(uvd[1]).astype(int)
        depth = uvd[2]
        if depth < depth_map[v,u]:
            depth_map[v,u] = depth
            
    min_depth_neighborhood = minimum_filter(depth_map, size=20)
    
    for i, uvd in enumerate(projected_points):
        u = np.round(uvd[0]).astype(int)
        v = np.round(uvd[1]).astype(int)
        depth = uvd[2]
        if depth <= min_depth_neighborhood[v,u] + 0.1: # 0.1m depth tolerance
            visibility_mask.append(True)
        else:
            visibility_mask.append(False)
            
    return np.array(visibility_mask)
    
def scalar_to_bgr(value):
    # Ensure the value is within the range [0, 1]
    value = max(0, min(1, value))

    # Interpolate between the colors [1, 0, 0] (red) at value 0, [0, 1, 0] (green) at value 0.5, and [0, 0, 1] (blue) at value 1
    if value < 0.5:
        blue = 1 - 2 * value
        green = 2 * value
        red = 0
    else:
        blue = 0
        green = 2 - 2 * value
        red = 2 * (value - 0.5)
    
    # color = 255.0 * np.array([blue, green, red])
    color = (int(255.0*blue), int(255.0*green), int(255.0*red))
    return color


T_l_c_final = np.genfromtxt(os.path.join(root, 'Final_T_l_c.txt'))

save_path_proj_imgs = os.path.join(root, 'Projections')
os.makedirs(save_path_proj_imgs, exist_ok=True)

# Max and min depth value for visualization color scaling:
maxDepth = 2.5
minDepth = 0.3

N_stamps = len(timestamp_dict)

for i, key in enumerate(timestamp_dict):

    img = cv2.imread(os.path.join(camera_path, key+'.png'),0)
    lidar = np.fromfile(os.path.join(lidar_path, timestamp_dict[key]+'.bin'), dtype=np.float32).reshape(-1, 4)

    img_undist = cv2.undistort(img, Cmat, dist_coeffs, None, Cmat)
    disp_img = cv2.cvtColor(img_undist,cv2.COLOR_GRAY2BGR)

    points3D_lidar = lidar[:,:3]
    points3D_lidar_H = np.column_stack((points3D_lidar, np.ones(points3D_lidar.shape[0])))
    points3D = (T_l_c_final @ points3D_lidar_H.T).T

    points2D = projectToImg(points3D[:,:3])
    visible_idxs = check_visibility(points2D)
    visible_pts = points2D[visible_idxs,:]

    for pt2D in visible_pts:

        scalar = (pt2D[2] - minDepth) / (maxDepth - minDepth)
        color = scalar_to_bgr(scalar)
        center = tuple(pt2D[:2].astype(int))
        disp_img = cv2.circle(disp_img, center, 2, color, -1)

    save_name = key+'.png'
    cv2.imwrite(os.path.join(save_path_proj_imgs, save_name), disp_img)

    progress = float(i) / float(N_stamps) * 100
    print('Progress:', progress)

plt.show()

### Pose estimation evaluation using the obtained extrinsic transformation

Compare observed LiDAR poses vs estimated LiDAR poses using thermal camera poses and the extrinsic transformation. Note that the error also contains noise and inconsistencies due to the computation of the pose observations.

In [ ]:
# %matplotlib qt

import matplotlib.pyplot as plt

def getRMSE(diffs):

    diffs_sqrd = diffs**2
    return np.sqrt(np.mean(diffs_sqrd))

T_l_c_final = np.genfromtxt(os.path.join(root, 'Final_T_l_c.txt'))
T_c_l_final = transposeT(T_l_c_final)

poses3D = np.genfromtxt(os.path.join(root, 'poses3D.csv'), delimiter=',', skip_header=1)
poses2D = np.genfromtxt(os.path.join(root, 'poses2D.csv'), delimiter=',', skip_header=1)

poses2D_dict = {}

for pose2D in poses2D:
    key = pose2D[0].astype(int).astype(str)
    value = pose2D[2:].reshape(4,4)
    poses2D_dict[key] = value

errors = []
N_t_below = 0
err_angles = []
diffs = []
t_threshold = 0.15 #0.1
r_threshold = 9.0 #7.0
N_r_below = 0

for i in range(len(poses3D)):
    
    timestamp3D = poses3D[i,0].astype(int).astype(str)
    timestamp2D = inv_timestamp_dict[timestamp3D]

    pose2D = poses2D_dict[timestamp2D]
    pose3D = poses3D[i,1:].reshape(4,4)

    est_pose3D = transposeT(T_c_l_final @ transposeT(pose2D))
    diff_t = pose3D[:3,3] - est_pose3D[:3,3]
    error_t = np.linalg.norm(diff_t)

    # if error_t < 0.2:
    errors.append(error_t)
    diffs.append(diff_t)

    if error_t < t_threshold:
        N_t_below += 1

    R_err = est_pose3D[:3,:3] @ pose3D[:3,:3].T
    angle_error = np.arccos((np.trace(R_err) - 1) / 2) / np.pi * 180.0
    # if angle_error < 10.0:
    err_angles.append(angle_error)

    if angle_error < r_threshold:
        N_r_below += 1

diffs = np.array(diffs)
perc_t_below = float(N_t_below) / float(len(poses3D))
rmse_t = getRMSE(diffs)
rmse_t_per_axis = [getRMSE(diffs[:,0]), getRMSE(diffs[:,1]), getRMSE(diffs[:,2])]
print('RMSE translation per axis:', rmse_t_per_axis, 'm')
print('Percentage below threshold T:', perc_t_below)

err_angles = np.array(err_angles)
perc_r_below = float(N_r_below) / float(len(poses3D))
rmse_r = getRMSE(err_angles)
print('RMSE rotation:', rmse_r, 'deg')
print('Percentage below threshold R:', perc_r_below)

mean_e_t = np.mean(errors)
mean_e_r = np.mean(err_angles)

plt.figure(1)
ax = plt.plot(range(len(errors)), errors, '-b', linewidth=3)
plt.plot([0, len(errors)], [mean_e_t, mean_e_t], '--r', linewidth=3, label='Mean error = {:.4f} m'.format(mean_e_t))
plt.xlabel('Sample index')
plt.ylabel('Error [m]')
# plt.yscale('log')
plt.title('Translational RMSE: {:.4f} m'.format(rmse_t))
plt.grid(which='both')
plt.legend()
plt.tight_layout()
# plt.savefig(os.path.join(root,'trans_error'))

plt.figure(2)
ax = plt.plot(range(len(err_angles)), err_angles, '-b', linewidth=3)
plt.plot([0.0, len(err_angles)], [mean_e_r, mean_e_r], '--r', linewidth=3, label='Mean error = {:.3f} deg'.format(mean_e_r))
plt.xlabel('Sample index')
plt.ylabel('Error [deg]')
# plt.yscale('log')
plt.title('Rotational RMSE: {:.2f} degrees'.format(rmse_r))
plt.grid(which='both')
plt.legend()
plt.tight_layout()
# plt.savefig(os.path.join(root,'rot_error'))
plt.show()